In [1]:
import numpy as np
import os

In [2]:
def save_to_file(BASE_PATH, fileName, objectToWrite):
    with open(os.path.join(BASE_PATH, fileName), 'wb') as f:
        np.save(f, objectToWrite)

In [3]:
def load_from_file(BASE_PATH, fileName):
    with open(os.path.join(BASE_PATH, fileName), 'rb') as f:
        file = np.load(f)
    return file

## spacy_universal_sentence_encoder

Resources Consulted: https://spacy.io/universe/project/spacy-universal-sentence-encoder

In [1]:
import spacy_universal_sentence_encoder
import pandas as pd

In [3]:
english_train = pd.read_csv(os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'Data\english_train.csv'))
english_validation = pd.read_csv(os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), 'Data\english_validation.csv'))

In [7]:
ls_english_train = english_train['text'].to_list()
ls_english_validation = english_validation['text'].to_list()

In [11]:
nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')
def get_spacy_universal_sentence_encoder_embedding(sentence):
    doc = nlp(sentence)
    return doc.vector

In [18]:
# ls_english_train_spacy_universal_sentence_encoder_embedding = []
# for i, sentence in enumerate(ls_english_train):
#     ls_english_train_spacy_universal_sentence_encoder_embedding.append(get_spacy_universal_sentence_encoder_embedding(sentence))

In [54]:
# save_to_file(os.path.join(os.getcwd(),'Pre Computed Word Embeddings'), 'ls_english_train_spacy_universal_sentence_encoder_embedding', ls_english_train_spacy_universal_sentence_encoder_embedding)

In [5]:
ls_english_train_spacy_universal_sentence_encoder_embedding = load_from_file(os.path.join(os.getcwd(),'Pre Computed Word Embeddings'), 'ls_english_train_spacy_universal_sentence_encoder_embedding')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.layers import Input, Dense, Reshape,Embedding,dot
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from helper import build_dataset
import numpy as np

In [ ]:
raw_data = #add the data

corpus = raw_data.split()

In [ ]:
window_size = 2
vector_dim = 300
vocab_size = len(corpus)

data, count, dictionary, reverse_dictionary = build_dataset(corpus,vocab_size)

In [ ]:
couples, labels = skipgrams(data, vocab_size, window_size=window_size)
word_target, word_context = zip(*couples)
# word_target = np.array(word_target, dtype="int32")
# word_context = np.array(word_context, dtype="int32")
print(couples[:5], labels[:5])

In [ ]:
word_model = Sequential() 
word_model.add(Embedding(vocab_size, vector_dim, 
               input_length=1,name='embedding')) 
word_model.add(Reshape((vector_dim, ))) 

context_model = Sequential() 
context_model.add(Embedding(vocab_size, vector_dim, 
                  input_length=1,name='context')) 
context_model.add(Reshape((vector_dim,))) 

dot_product = dot([word_model.output, context_model.output], axes=1,
                  normalize=False,name='dotproduct') 
dot_product = Dense(1,activation="sigmoid")(dot_product) 

model = Model(inputs=[word_model.input, context_model.input], 
              outputs=dot_product,name='SGNS') 

model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop") 

In [ ]:
idx = np.random.randint(0, len(labels)-1)

target_input = np.array(word_target[idx],dtype='float32').reshape(1,)
context_input = np.array(word_context[idx],dtype='float32').reshape(1,)
training_label = np.array(labels[idx],dtype='float32').reshape(1,)

loss = model.train_on_batch([target_input, context_input], training_label)
print(f'Loss after one epoch is {loss:.2f}')

In [ ]:
word_model = Sequential()
word_model.add(Input(vocab_size))
word_model.add(Dense(vector_dim, activation = "linear")) 
word_model.add(Reshape((vector_dim, ))) 

context_model = Sequential()
context_model.add(Input(vocab_size))
context_model.add(Dense(vector_dim, activation = "linear")) 
context_model.add(Reshape((vector_dim,))) 

dot_product = dot([word_model.output, context_model.output], axes=1,
                  normalize=False,name='dotproduct') 

dot_product = Dense(3,activation="softmax")(dot_product)
model = Model(inputs=[word_model.input, context_model.input], 
              outputs=dot_product,name='Custom') 

model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="rmsprop") 

In [ ]:
idx = np.random.randint(0, len(labels)-1)

a = to_categorical(word_target,num_classes = (len(list(set(word_target)))+1))
b = to_categorical(word_context,num_classes = (len(list(set(word_target)))+1))
onehot_target = np.expand_dims(a[idx], axis = 0)
onehot_context = np.expand_dims(b[idx], axis = 0)
training_label = np.array(labels[idx],dtype='float32').reshape(1,)

loss = model.train_on_batch([onehot_target, onehot_context], training_label)
print(f'Loss after one epoch is {loss:.2f}')